In [1]:
# Initial imports
import os # Miscellaneous operating system interfaces (os.getenv)
import requests # requests.get
import pandas as pd
from dotenv import load_dotenv #Reads the key-value pair from .env file and adds them to environment variable.
from pathlib import Path
import json # lightweight data interchange format 
import matplotlib.pyplot as plt

%matplotlib inline
import FundamentalAnalysis as fa
import alpaca_trade_api as tradeapi


Bad key "text.kerning_factor" on line 4 in
C:\Users\Khemero\anaconda3\envs\pyvizenv\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
http://github.com/matplotlib/matplotlib/blob/master/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
# Load .env enviroment variables
load_dotenv()
from pathlib import Path
env_path = Path('C:/Users/Khemero/.env')

In [3]:
# Set FA API key
api_key = os.getenv("FA")

In [4]:
# Set Alpaca API key and secret
alpaca_api_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

# Create the Alpaca API object
alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")

In [5]:
# Set the file path
sp_data = Path("Resources/sp.csv")

In [6]:
# Read whale returns csv
# Read the CSVs and set the `date` column as a datetime index to the DataFrame
sp_df = pd.read_csv(sp_data, index_col="Date", infer_datetime_format=True, parse_dates=True)
sp_df.sort_index(ascending = True, inplace = True)

In [7]:
# Drop nulls
sp_df = sp_df.dropna()

In [8]:
# master_list = []
# for i in target_items:
#         data_dict = {}
#         data_dict['marketplace'] = 'ebay'
#         data_dict['date_retrieved'] = date.today()
#         data_dict['link'] = i.get('a',{'href'})
#         master_list.append(data_dict)


In [9]:
# Format current date as ISO format
today = pd.Timestamp("2020-07-31", tz="America/New_York").isoformat() 
start = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat() 
end = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat() # last year

# Set the tickers
tickers = ["TSLA", "NIO", "AAPL", "PFE", "MSFT", "MRNA", "ZM", "AMZN", "NNDM", "CRM", "NVDA"]
dict = {'stock':'', 'segment':''}  
marketcaps = {}
segments = {}
for stock in tickers:
    profile = fa.profile(stock, api_key) 
    try: 
        marketcaps[stock] = int(profile.iloc[22])
        segment = str(profile.iloc[26])[5:]
        segments[stock] = segment[:-28]
    except: continue
    

## Create the shares DataFrame with `tickers` as index
#df_shares = pd.DataFrame(shares_data, index=tickers)

# Set timeframe to '1D' for Alpaca API
timeframe = "1D"

# Get current closing prices for SPY and AGG
df_portfolio = alpaca.get_barset(
    tickers,
    timeframe,
    start = today,
    end = today
).df

In [10]:
# # Format current date as ISO format
# today = pd.Timestamp("2020-07-31", tz="America/New_York").isoformat() 
# start = pd.Timestamp("2019-01-01", tz="America/New_York").isoformat() #1/1/2019?
# end = pd.Timestamp("2020-12-31", tz="America/New_York").isoformat() #31/12/2019?

# # Set the tickers
# tickers = ["TSLA", "NIO", "AAPL", "PFE", "MSFT", "MRNA", "ZM", "AMZN", "NNDM", "CRM", "NVDA"]
# dict = {'stock':'', 'segment':''}  
# marketcaps = {}
# segments = {}
# for stock in tickers:
#     profile = fa.profile(stock, api_key) 
#     try: 
#         marketcaps[stock] = int(profile.iloc[22])
#         segment = str(profile.iloc[26])[5:]
#         segments[stock] = segment[:-28]
#     except: continue
    

# ## Create the shares DataFrame with `tickers` as index
# #df_shares = pd.DataFrame(shares_data, index=tickers)

# # Set timeframe to '1D' for Alpaca API
# timeframe = "1D"

# # Get current closing prices for SPY and AGG
# df_portfolio = alpaca.get_barset(
#     tickers,
#     timeframe,
#     start = today,
#     end = today
# ).df

In [11]:
marketcaps_df = pd.DataFrame(marketcaps.items(), columns=['stock', 'marketcap'])
segments_df = pd.DataFrame(segments.items(), columns=['stock', 'marketcap']).rename(columns={
    "marketcap":"sector"
})

In [12]:
marketcaps_df.head(11)

,stock,marketcap
0,TSLA,658791200000
1,NIO,72832279000
2,AAPL,2153362880000
3,PFE,209440506000
4,MSFT,1652649750000
5,MRNA,55490413000
6,ZM,116121706000
7,AMZN,1606431010000
8,NNDM,991197310
9,CRM,208098443000


In [13]:
segments_df.head(11)

,stock,sector
0,TSLA,Consumer Cyclical
1,NIO,Consumer Cyclical
2,AAPL,Technology
3,PFE,Healthcare
4,MSFT,Technology
5,MRNA,Healthcare
6,ZM,Communication Services
7,AMZN,Consumer Cyclical
8,NNDM,Technology
9,CRM,Technology


In [14]:
segments = segments_df.drop(columns=[
    "stock", 
])

In [15]:
joined_df = pd.concat([marketcaps_df, segments], axis="columns", join="inner")
joined_df.head(12)

,stock,marketcap,sector
0,TSLA,658791200000,Consumer Cyclical
1,NIO,72832279000,Consumer Cyclical
2,AAPL,2153362880000,Technology
3,PFE,209440506000,Healthcare
4,MSFT,1652649750000,Technology
5,MRNA,55490413000,Healthcare
6,ZM,116121706000,Communication Services
7,AMZN,1606431010000,Consumer Cyclical
8,NNDM,991197310,Technology
9,CRM,208098443000,Technology


In [16]:
sp_df.head()

,Open,High,Low,Close,Volume
Date,,,,,
2019-01-02 16:00:00,2476.96,2519.49,2467.47,2510.03,2233765369
2019-01-03 16:00:00,2491.92,2493.14,2443.96,2447.89,2754752741
2019-01-04 16:00:00,2474.33,2538.07,2474.33,2531.94,2661709794
2019-01-07 16:00:00,2535.61,2566.16,2524.56,2549.69,2526552370
2019-01-08 16:00:00,2568.11,2579.82,2547.56,2574.41,2458286256


In [17]:
profile = fa.profile("AAPL", api_key)
profile

,0
address,1 Apple Park Way
beta,1.33758
ceo,Mr. Timothy Cook
changes,-2.045
cik,0000320193
city,Cupertino
companyName,Apple Inc
country,US
currency,USD
cusip,037833100


In [18]:
# # Collect general company information
# profile = fa.profile(ticker, api_key)

# # Collect recent company quotes
# quotes = fa.quote(ticker, api_key)

# # Collect market cap and enterprise value
# entreprise_value = fa.enterprise(ticker, api_key)

# # Show recommendations of Analysts
# ratings = fa.rating(ticker, api_key)

# # Obtain DCFs over time
# dcf_annually = fa.discounted_cash_flow(ticker, api_key, period="annual")
# dcf_quarterly = fa.discounted_cash_flow(ticker, api_key, period="quarter")

# # Collect the Balance Sheet statements
# balance_sheet_annually = fa.balance_sheet_statement(ticker, api_key, period="annual")
# balance_sheet_quarterly = fa.balance_sheet_statement(ticker, api_key, period="quarter")

# # Collect the Income Statements
# income_statement_annually = fa.income_statement(ticker, api_key, period="annual")
# income_statement_quarterly = fa.income_statement(ticker, api_key, period="quarter")

# # Collect the Cash Flow Statements
# cash_flow_statement_annually = fa.cash_flow_statement(ticker, api_key, period="annual")
# cash_flow_statement_quarterly = fa.cash_flow_statement(ticker, api_key, period="quarter")

# # Show Key Metrics
# key_metrics_annually = fa.key_metrics(ticker, api_key, period="annual")
# key_metrics_quarterly = fa.key_metrics(ticker, api_key, period="quarter")

# # Show a large set of in-depth ratios
# financial_ratios_annually = fa.financial_ratios(ticker, api_key, period="annual")
# financial_ratios_quarterly = fa.financial_ratios(ticker, api_key, period="quarter")

# # Show the growth of the company
# growth_annually = fa.financial_statement_growth(ticker, api_key, period="annual")
# growth_quarterly = fa.financial_statement_growth(ticker, api_key, period="quarter")

# # Download general stock data
# stock_data = fa.stock_data(ticker, period="ytd", interval="1d")

# # Download detailed stock data
# stock_data_detailed = fa.stock_data_detailed(ticker, api_key, begin="2000-01-01", end="2020-01-01")
